In [1]:
df=sqlContext.sql(" SELECT * FROM  customer")

In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
df.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [4]:
df.columns

Out[ 7 ]: 
['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [5]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer
assembler=VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [6]:
output=assembler.transform(df)
final=output.select('features','Churn')

In [7]:
logreg=LogisticRegression(featuresCol='features',labelCol='Churn')

In [8]:
train,test=final.randomSplit([0.7,0.3])

In [9]:
fitt=logreg.fit(train)

In [10]:
t=fitt.summary

In [11]:
t.predictions.describe().show()

+-------+-------------------+------------------+
summary| Churn| prediction|
+-------+-------------------+------------------+
 count| 628| 628|
 mean|0.17993630573248406|0.1449044585987261|
 stddev|0.38444059786791757|0.3522850770984735|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+------------------+

In [12]:
k=fitt.evaluate(test)

In [13]:
k.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|Churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[26.0,8787.39,1.0...| 1|[0.47660887932965...|[0.61694679190198...| 0.0|
[27.0,8628.8,1.0,...| 0|[5.37031213935061...|[0.99536886836849...| 0.0|
[28.0,11128.95,1....| 0|[4.14683375037325...|[0.98443179234724...| 0.0|
[28.0,11204.23,0....| 0|[1.95065599441186...|[0.87551815370932...| 0.0|
[28.0,11245.38,0....| 0|[3.53758454038463...|[0.97173845219949...| 0.0|
[29.0,9617.59,0.0...| 0|[4.35538995543547...|[0.98732527830388...| 0.0|
[29.0,10203.18,1....| 0|[3.64014949439074...|[0.97442293778243...| 0.0|
[29.0,11274.46,1....| 0|[4.55031763793287...|[0.98954657997221...| 0.0|
[29.0,13240.01,1....| 0|[6.82797369300820...|[0.99891812124089...| 0.0|
[30.0,7960.64,1.0...| 1|[3.30585144505529...|[0.96462900508256...| 0.0|
[30.0,8403.78,1.0...| 0|[5.99055634984172...|[0.99750397420975...| 0.0|
[30.0,10183.98,1....| 0|[2.82463930789949...|[0.94399285407585...| 0.0|
[30.0,10744.14,1....| 1|[1.44322876960408...|[0.80895414701007...| 0.0|
[30.0,12788.37,0....| 0|[2.57268364431727...|[0.92908271966287...| 0.0|
[31.0,8829.83,1.0...| 0|[4.43918898256170...|[0.98833223479640...| 0.0|
[31.0,12264.68,1....| 0|[3.42431667492545...|[0.96845590898728...| 0.0|
[32.0,9036.27,0.0...| 0|[-0.5716987254315...|[0.36084494529139...| 1.0|
[32.0,9472.72,1.0...| 0|[4.30884931838036...|[0.98672945948503...| 0.0|
[32.0,11540.86,0....| 0|[7.02539735441788...|[0.99911177544854...| 0.0|
[32.0,12403.6,0.0...| 0|[5.65587920423681...|[0.99651528843031...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
eval=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

In [15]:
p=eval.evaluate(k.predictions)

In [16]:
p

Out[ 25 ]: 0.7333525014376079

In [17]:
finmode=logreg.fit(final)

In [18]:
da=sqlContext.sql(" SELECT * FROM  new")

In [19]:
da.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)

In [20]:
test=assembler.transform(da)

In [21]:
test.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- features: vector (nullable = true)

In [22]:
finalresults=finmode.transform(test)

In [23]:
finalresults.select('Company','prediction').show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+